# Abstract

In [ ]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

# Environment

In [ ]:
import numpy as np

In [ ]:
from jwst.associations import (
    AssociationRegistry,
    generate
)

In [ ]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only
)

# Library

# Main

## Level2

In [ ]:
rules = registry_level2_only()

### Get all the spectral EXP_TYPEs

In [ ]:
asn_lv2spec = rules['Asn_Lv2Spec']()

In [ ]:
spec_exptypes = asn_lv2spec.constraints['exp_type']['value'].split('|')

### MIRI spectra without backgrounds
Formal test: test_level2_basics

In [ ]:
pool = combine_pools('data/pool_007_spec_miri.csv')

In [ ]:
asns, orphaned = generate(pool, rules)

In [ ]:
assert len(asns) == 11

In [ ]:
asns

In [ ]:
is_spec = np.array([
    row['EXP_TYPE'] in spec_exptypes
    for row in pool
])

In [ ]:
pool[is_spec]['EXP_TYPE']

## Try with a very simple pool
Formal test: test_level2_background

In [ ]:
pool = combine_pools('data/pool_011_spec_miri_lv2bkg_lrs.csv')

In [ ]:
asns, orphaned = generate(pool, rules)

In [ ]:
assert len(asns) == 3

In [ ]:
asns

## Try with all MIRI data

In [ ]:
pool = combine_pools('data/pool_009_spec_miri_lv2bkg.csv')

In [ ]:
asns, orphaned = generate(pool, rules)

In [ ]:
assert len(asns) == 13

In [ ]:
invalid_asns =[
    asn
    for asn in asns
    if len(asn['members']) == 1 and asn['asn_rule'] == 'Asn_Lv2SpecBkg'
]

In [ ]:
assert len(invalid_asns) == 0

In [ ]:
is_spec = np.array([
    row['EXP_TYPE'] in spec_exptypes
    for row in pool
])

In [ ]:
pool[is_spec]['EXP_TYPE']

In [ ]:
is_lv2_asn = [
    asn
    for asn in asns
    if asn['asn_rule'] == 'Asn_Lv2Spec'
]

In [ ]:
len(is_lv2_asn)

In [ ]:
is_lv2bkg_asn = [
    asn
    for asn in asns
    if asn['asn_rule'] == 'Asn_Lv2SpecBkg'
]

In [ ]:
len(is_lv2bkg_asn)

In [ ]:
is_lv2bkg_asn

In [ ]:
is_bkg = np.array([
    row['BACKGROUND'] != 'NULL'
    for row in pool
])

In [ ]:
pool[is_bkg]

### Test NIRSpec

In [ ]:
pool = combine_pools('data/pool_010_spec_nirspec_lv2bkg.csv')

In [ ]:
asns, orphaned = generate(pool, rules)

In [ ]:
assert len(asns) == 12

In [ ]:
asns

In [ ]:
invalid_asns =[
    asn
    for asn in asns
    if len(asn['members']) == 1 and asn['asn_rule'] == 'Asn_Lv2SpecBkg'
]

In [ ]:
assert len(invalid_asns) == 0

## test_generate confirmation

In [ ]:
rules = AssociationRegistry()

In [ ]:
pool = combine_pools('data/mega_pool.csv')

In [ ]:
asns, orphanged = generate(pool, rules)

In [ ]:
len(asns)

In [ ]:
asns

## test_associations confirmation

### Generic constraint

In [ ]:
generic_constraint = {
    'obs_id': {
        'value': 'V99009001001P0000000002101',
        'inputs': ['OBS_ID']
    }
}

In [ ]:
rules = AssociationRegistry(global_constraints=generic_constraint)

In [ ]:
pool = combine_pools('data/mega_pool.csv')

In [ ]:
asns, orphaned = generate(pool, rules)

In [ ]:
len(asns)

In [ ]:
pool.show_in_browser()

In [ ]:
%pwd

In [ ]:
rules

In [ ]:
pool_one = pool[pool['OBS_ID'] == 'V99009001001P0000000002101']

In [ ]:
pool_one

In [ ]:
asns, orphaned = generate(pool_one, rules)

In [ ]:
asns

In [ ]:
asn = rules['Asn_Image'](pool_one[0])

In [ ]:
asn

In [ ]:
asn.validity